In [ ]:
# Google Drive 마운트 & 경로 설정
from google.colab import drive
drive.mount('/content/drive')

import os, glob, shutil

OUT_DIR = "/content/outputs"                       # 로컬 결과/체크포인트 폴더
DRIVE_DIR = "/content/drive/MyDrive/dcgan_checkpoints"  # 드라이브 백업 폴더
os.makedirs(OUT_DIR, exist_ok=True)
os.makedirs(DRIVE_DIR, exist_ok=True)

Mounted at /content/drive


### 데이터 불러오기

In [ ]:
from google.colab import files
files.upload()  # kaggle.json 업로드 창 표시
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [ ]:
#!/bin/bash
!kaggle datasets download deewakarchakraborty/portrait-paintings

Dataset URL: https://www.kaggle.com/datasets/deewakarchakraborty/portrait-paintings
License(s): CC0-1.0
 99% 222M/223M [00:00<00:00, 710MB/s] 
100% 223M/223M [00:00<00:00, 749MB/s]


### 기본 설정

In [ ]:
!unzip -q portrait-paintings.zip

In [ ]:
import os, glob, random, math, time
from PIL import Image
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import transforms
from torchvision.utils import save_image, make_grid

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("DEVICE:", DEVICE)

DEVICE: cuda


In [ ]:
# 재현성
SEED = 2025
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED);
if DEVICE.type == "cuda":
    torch.cuda.manual_seed_all(SEED)

In [ ]:
# 하이퍼파라미터
image_size = 64        # DCGAN 기본 사이즈(빠름). 더 선명하게는 128도 가능
nc = 3                 # 채널 수(RGB=3)
nz = 100               # 잠재 벡터 차원
ngf = 64               # G의 feature map 크기
ndf = 64               # D의 feature map 크기
batch_size = 64
num_epochs = 100       # 결과 보고 늘리기
lr = 0.0002
beta1 = 0.5            # Adam beta1(DCGAN 권장값)

### 데이터 준비

In [ ]:
class PortraitFolder(Dataset):
    def __init__(self, root_dir, transform=None, exts=("*.jpg","*.jpeg","*.png","*.webp","*.bmp")):
        self.root_dir = root_dir
        self.transform = transform
        files = []
        # 지정된 확장자별로 파일 경로 수집 (현재 폴더 + 하위 폴더까지 검색)
        for ext in exts:
            files.extend(glob.glob(os.path.join(root_dir, ext)))  # 루트 폴더 1단계
            files.extend(glob.glob(os.path.join(root_dir, "**", ext), recursive=True)) # 재귀적으로 하위 폴더까지
        # 중복 제거 + 정렬
        self.files = sorted(list(set(files)))

        # 파일이 정말 이미지인지 간단 필터링(깨진 파일 방지)
        ok_files = []
        for f in self.files:
            try:
                with Image.open(f) as im:
                    im.verify() # 이미지 헤더만 검사 (실제 로딩은 안 함)
                ok_files.append(f)
            except Exception:
                pass # 열리지 않거나 깨진 파일은 무시
        self.files = ok_files

        # 유효한 이미지가 하나도 없으면 에러 발생
        if len(self.files) == 0:
            raise RuntimeError(f"No images found under: {root_dir}")

        print(f"Found {len(self.files)} images.") # 최종 이미지 개수 출력

    def __len__(self):
        return len(self.files) # 전체 이미지 개수 반환

    def __getitem__(self, idx):
        # index에 해당하는 이미지 파일 로드
        path = self.files[idx]
        img = Image.open(path).convert("RGB") # RGB로 통일 (흑백/투명 채널 제거)
        if self.transform:
            img = self.transform(img) # 전처리(transform) 적용
        return img

# 전처리
tfm = transforms.Compose([
    transforms.Resize(image_size), # 짧은 변을 image_size로 리사이즈
    transforms.CenterCrop(image_size), # 중앙에서 image_size x image_size 크기로 잘라내기
    transforms.ToTensor(), # [0,255] → [0,1] 범위의 Tensor로 변환
    transforms.Normalize([0.5]*3, [0.5]*3),  # 평균 0.5, 표준편차 0.5로 정규화 → [-1,1] 범위
])

# Dataset / DataLoader 생성
ds = PortraitFolder("/content/Images", transform=tfm) # 이미지 폴더 Dataset 생성
dl = DataLoader(ds, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True if DEVICE.type=="cuda" else False, drop_last=True)

Found 5734 images.


### DCGAN 모델 정의
- Generator: ConvTranspose2d → BN → ReLU 반복, 마지막 Tanh
- Discriminator: Conv2d → (BN) → LeakyReLU 반복, 마지막엔 로짓 1개

In [ ]:
# 가중치 초기화(DCGAN 권장)
def weights_init_dcgan(m):
    classname = m.__class__.__name__.lower() # 모듈의 클래스 이름 확인 (예: Conv2d, BatchNorm2d 등)

    # 1) Convolution 계층 초기화
    if "conv" in classname:
        # Conv 계층의 weight를 평균 0, 표준편차 0.02인 정규분포로 초기화
        nn.init.normal_(m.weight.data, 0.0, 0.02)
        # bias가 있다면 0으로 초기화
        if getattr(m, "bias", None) is not None and m.bias is not None:
            nn.init.zeros_(m.bias.data)

    # 2) BatchNorm 계층 초기화
    elif "batchnorm" in classname:
        # weight(γ)를 평균 1.0, 표준편차 0.02인 정규분포로 초기화 → 스케일 파라미터
        if getattr(m, "weight", None) is not None:
            nn.init.normal_(m.weight.data, 1.0, 0.02)
        # bias(β)는 0으로 초기화 → 이동 파라미터
        if getattr(m, "bias", None) is not None:
            nn.init.zeros_(m.bias.data)

In [ ]:
# Generator
class Generator(nn.Module):
    def __init__(self, nz=100, ngf=64, nc=3):
        super().__init__()
        self.main = nn.Sequential(
            # 입력: (nz, 1, 1) → 첫 입력은 랜덤 노이즈 z (100차원 벡터)
            nn.ConvTranspose2d(nz, ngf*8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf*8),
            nn.ReLU(True),
            # upsampling: (ngf*8, 4, 4) → (ngf*4, 8, 8)
            nn.ConvTranspose2d(ngf*8, ngf*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf*4),
            nn.ReLU(True),
            # (ngf*4, 8, 8) → (ngf*2, 16, 16)
            nn.ConvTranspose2d(ngf*4, ngf*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf*2),
            nn.ReLU(True),
            # (ngf*2, 16, 16) → (ngf, 32, 32)
            nn.ConvTranspose2d(ngf*2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # (ngf, 32, 32) → (nc=3, 64, 64) 최종 RGB 이미지
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh(),  # 출력 범위 [-1, 1]
        )
    def forward(self, z):
        return self.main(z) # (B, 3, 64, 64)

In [ ]:
# Discriminator
class Discriminator(nn.Module):
    def __init__(self, nc=3, ndf=64):
        super().__init__()
        self.main = nn.Sequential(
            # 입력: (3, 64, 64) → 첫 레이어에서 특징 추출
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # (ndf, 32, 32) → (ndf*2, 16, 16)
            nn.Conv2d(ndf, ndf*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf*2),
            nn.LeakyReLU(0.2, inplace=True),
            # (ndf*2, 16, 16) → (ndf*4, 8, 8)
            nn.Conv2d(ndf*2, ndf*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf*4),
            nn.LeakyReLU(0.2, inplace=True),
            # (ndf*4, 8, 8) → (ndf*8, 4, 4)
            nn.Conv2d(ndf*4, ndf*8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf*8),
            nn.LeakyReLU(0.2, inplace=True),
            # (ndf*8, 4, 4) → (1, 1, 1) → 진짜/가짜 판별 스칼라 값
            nn.Conv2d(ndf*8, 1, 4, 1, 0, bias=False),
            # Sigmoid 없음 (PyTorch에서 BCEWithLogitsLoss가 자동으로 Sigmoid 내장)
        )
    def forward(self, x):
        out = self.main(x)        # 출력: (B, 1, 1, 1)
        return out.view(-1)       # reshape → (B,)

In [ ]:
# 모델 생성 + 초기화
netG = Generator(nz, ngf, nc).to(DEVICE)
netD = Discriminator(nc, ndf).to(DEVICE)
netG.apply(weights_init_dcgan)
netD.apply(weights_init_dcgan)

Discriminator(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
  )
)

### 손실 함수 & 옵티마이저 설정

In [ ]:
# 손실/옵티마이저
criterion = nn.BCEWithLogitsLoss().to(DEVICE)
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

In [ ]:
# 고정 노이즈(진행 모니터)
fixed_noise = torch.randn(64, nz, 1, 1, device=DEVICE)

### 학습 루프

- D는 real(=1)과 fake(=0)를 구분하도록 학습
- G는 D를 속이도록(=1로 분류되도록) 학습
- 매 epoch마다 샘플 이미지 저장

In [ ]:
# 학습
iters = 0 # 전체 학습 step 수를 기록할 카운터
log_interval = 100 # 로그 출력 주기(몇 iteration마다 결과 보여줄지)
os.makedirs(OUT_DIR, exist_ok=True) # 출력 폴더 생성 (없으면 새로 만듦)

for epoch in range(1, num_epochs+1):
    netG.train(); netD.train() # 학습 모드로 전환 (BN/Dropout 동작 차이 있음)
    pbar = tqdm(dl, desc=f"[Epoch {epoch}/{num_epochs}]") # 진행 상태 표시
    for real in pbar:
        # (1) Discriminator 학습
        real = real.to(DEVICE) # 진짜 이미지 배치 (GPU로)
        bsz = real.size(0) # 현재 배치 크기
        real_label = torch.ones(bsz, device=DEVICE) # 진짜 라벨 = 1
        fake_label = torch.zeros(bsz, device=DEVICE) # 가짜 라벨 = 0

        optimizerD.zero_grad(set_to_none=True) # D의 기울기 초기화
        d_real = netD(real) # 진짜 이미지 판별
        d_real_loss = criterion(d_real, real_label) # 진짜 → 1이 되도록 BCE Loss

        noise = torch.randn(bsz, nz, 1, 1, device=DEVICE) # 랜덤 노이즈
        fake = netG(noise).detach() # Generator로 가짜 이미지 생성 (detach → G 업데이트 X)
        d_fake = netD(fake) # D가 가짜 이미지 판별
        d_fake_loss = criterion(d_fake, fake_label) # 가짜 → 0이 되도록 BCE Loss

        d_loss = d_real_loss + d_fake_loss # D의 최종 손실 (진짜+가짜)
        d_loss.backward() # 역전파
        optimizerD.step() # D의 파라미터 업데이트

        # (2) Generator 학습
        optimizerG.zero_grad(set_to_none=True) # G의 기울기 초기화
        noise = torch.randn(bsz, nz, 1, 1, device=DEVICE) # 새로운 노이즈
        gen = netG(noise) # G로 가짜 이미지 생성
        d_gen = netD(gen) # D가 그 이미지를 판별
        g_loss = criterion(d_gen, real_label) # G는 D를 속여서 "진짜=1"로 만들고 싶음
        g_loss.backward() # 역전파
        optimizerG.step() # G의 파라미터 업데이트

        # (3) 로그 출력
        iters += 1
        if iters % log_interval == 0:
            pbar.set_postfix({
                "D_real": f"{d_real_loss.item():.3f}",
                "D_fake": f"{d_fake_loss.item():.3f}",
                "D": f"{d_loss.item():.3f}",
                "G": f"{g_loss.item():.3f}"
            })

    # (4) 에폭마다 샘플 이미지 저장
    netG.eval() # 평가 모드 (BN/Dropout 고정)
    with torch.no_grad():
        fakes = netG(fixed_noise).cpu() # 고정 노이즈 입력으로 가짜 이미지 생성
        save_image((fakes*0.5+0.5).clamp(0,1), # [-1,1] → [0,1] 범위로 복원
                   fp=os.path.join(OUT_DIR, f"samples_epoch_{epoch:03d}.png"),
                   nrow=8) # 8x8 그리드로 저장

    # 체크포인트 저장
    torch.save({
        "epoch": epoch,
        "netG": netG.state_dict(), # Generator 가중치
        "netD": netD.state_dict(), # Discriminator 가중치
        "optimizerG": optimizerG.state_dict(), # G 옵티마이저 상태
        "optimizerD": optimizerD.state_dict(), # D 옵티마이저 상태
        "nz": nz, "ngf": ngf, "ndf": ndf, "nc": nc, "image_size": image_size,
    }, os.path.join(OUT_DIR, f"ckpt_{epoch:03d}.pt"))

print("학습 완료! 샘플 이미지는", OUT_DIR, "에 저장되었습니다.")

[Epoch 10/10]: 100%|██████████| 89/89 [00:17<00:00,  5.08it/s]


학습 완료! 샘플 이미지는 /content/outputs 에 저장되었습니다.


### 최신 체크포인트를 드라이브로 백업

In [ ]:
# 로컬 OUT_DIR의 가장 최신 ckpt를 드라이브로 복사
ckpts_local = sorted(glob.glob(os.path.join(OUT_DIR, "ckpt_*.pt")))
if not ckpts_local:
    raise RuntimeError("OUT_DIR에 ckpt_*.pt가 없습니다. 먼저 학습을 실행하세요.")
last_ckpt_local = ckpts_local[-1]
dst = os.path.join(DRIVE_DIR, os.path.basename(last_ckpt_local))
shutil.copy2(last_ckpt_local, dst)
print("드라이브 백업 완료:", dst)

드라이브 백업 완료: /content/drive/MyDrive/dcgan_checkpoints/ckpt_010.pt


### 드라이브에서 체크포인트 불러와 이어 학습

In [ ]:
# 드라이브 최신 ckpt 로드 & 이어 학습
import torch, glob, os
from tqdm import tqdm

# 1) 최신 ckpt 선택
ckpts_on_drive = sorted(glob.glob(os.path.join(DRIVE_DIR, "ckpt_*.pt")))
if not ckpts_on_drive:
    raise RuntimeError("DRIVE_DIR에 ckpt_*.pt가 없습니다.")
ckpt_path = ckpts_on_drive[-1]
print("불러올 체크포인트:", ckpt_path)

# 2) 로드 & 복구 (모델/옵티마이저는 기존 정의 사용)
ckpt = torch.load(ckpt_path, map_location=DEVICE)

# (모델 구조 하이퍼파라미터 동기화 — 혹시 달라졌다면 아래 값으로 덮어쓰기)
nz  = ckpt.get("nz", nz)
ngf = ckpt.get("ngf", ngf)
ndf = ckpt.get("ndf", ndf)
nc  = ckpt.get("nc", nc)
image_size = ckpt.get("image_size", image_size)

# 모델/옵티마이저 state 로드
netG.load_state_dict(ckpt["netG"])
netD.load_state_dict(ckpt["netD"])
if "optimizerG" in ckpt and "optimizerD" in ckpt:
    optimizerG.load_state_dict(ckpt["optimizerG"])
    optimizerD.load_state_dict(ckpt["optimizerD"])
    print("옵티마이저까지 복원 완료")
else:
    print("옵티마이저 상태 없음 → 옵티마이저는 새로 시작")

resume_from_epoch = ckpt.get("epoch", 0)
print(f"{resume_from_epoch} 에폭 이후부터 이어서 학습합니다.")

# 3) 이어서 k에폭만큼 더 학습
k = 10  # 추가 학습할 에폭 수 (원하는 숫자로 변경)
start_ep = resume_from_epoch + 1
end_ep   = resume_from_epoch + k

for epoch in range(start_ep, end_ep+1):
    netG.train(); netD.train()
    pbar = tqdm(dl, desc=f"[Resume Epoch {epoch}]")
    for real in pbar:
        real = real.to(DEVICE)
        bsz = real.size(0)
        real_label = torch.ones(bsz, device=DEVICE)
        fake_label = torch.zeros(bsz, device=DEVICE)

        # D 업데이트
        optimizerD.zero_grad(set_to_none=True)
        d_real = netD(real)
        d_real_loss = criterion(d_real, real_label)
        noise = torch.randn(bsz, nz, 1, 1, device=DEVICE)
        fake = netG(noise).detach()
        d_fake = netD(fake)
        d_fake_loss = criterion(d_fake, fake_label)
        d_loss = d_real_loss + d_fake_loss
        d_loss.backward()
        optimizerD.step()

        # G 업데이트
        optimizerG.zero_grad(set_to_none=True)
        noise = torch.randn(bsz, nz, 1, 1, device=DEVICE)
        gen = netG(noise)
        d_gen = netD(gen)
        g_loss = criterion(d_gen, real_label)
        g_loss.backward()
        optimizerG.step()

    # 샘플/체크포인트 저장 (에폭 번호를 이어서 저장)
    netG.eval()
    with torch.no_grad():
        fakes = netG(fixed_noise).cpu()
        save_image((fakes*0.5+0.5).clamp(0,1),
                   fp=os.path.join(OUT_DIR, f"samples_epoch_{epoch:03d}.png"),
                   nrow=8)

    torch.save({
        "epoch": epoch,
        "netG": netG.state_dict(),
        "netD": netD.state_dict(),
        "optimizerG": optimizerG.state_dict(),
        "optimizerD": optimizerD.state_dict(),
        "nz": nz, "ngf": ngf, "ndf": ndf, "nc": nc, "image_size": image_size,
    }, os.path.join(OUT_DIR, f"ckpt_{epoch:03d}.pt"))

print("Resume 학습 완료!")

불러올 체크포인트: /content/drive/MyDrive/dcgan_checkpoints/ckpt_010.pt
옵티마이저까지 복원 완료
10 에폭 이후부터 이어서 학습합니다.


[Resume Epoch 11]:  56%|█████▌    | 50/89 [00:11<00:08,  4.41it/s]


KeyboardInterrupt: 

### 마지막 체크포인트만 드라이브에 다시 백업

In [ ]:
# 가장 최신 로컬 ckpt를 드라이브에 복사
ckpts_local = sorted(glob.glob(os.path.join(OUT_DIR, "ckpt_*.pt")))
last_ckpt_local = ckpts_local[-1]
dst = os.path.join(DRIVE_DIR, os.path.basename(last_ckpt_local))
shutil.copy2(last_ckpt_local, dst)
print("드라이브 백업 완료:", dst)

In [ ]:
# GIF 만들기 (samples_epoch_XXX.png -> dcgan_training.gif)
import os, glob
from PIL import Image
from IPython.display import Image as DispImage, display
from google.colab import files

out_dir = "/content/outputs"
gif_path = os.path.join(out_dir, "dcgan_training.gif")

# 에폭 순서대로 정렬
frames = sorted(glob.glob(os.path.join(out_dir, "samples_epoch_*.png")))

if len(frames) == 0:
    raise RuntimeError("outputs 폴더에 samples_epoch_*.png가 없어요. 학습 루프가 이미지를 저장했는지 확인하세요!")

# 첫 프레임 기준으로 GIF 저장
imgs = [Image.open(p).convert("RGB") for p in frames]
imgs[0].save(
    gif_path,
    save_all=True,
    append_images=imgs[1:],
    duration=500,   # 프레임 간 시간(ms). 더 빠르게=작게, 느리게=크게
    loop=0          # 0이면 무한 반복
)

print(f"GIF 저장 완료: {gif_path}")
display(DispImage(filename=gif_path))  # 노트북에 표시
files.download(gif_path)               # 로컬로 다운로드

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import cv2
import os, glob
from IPython.display import Video, display
from google.colab import files

out_dir = "/content/outputs"
video_path = os.path.join(out_dir, "dcgan_training.mp4")

# 에폭 이미지 경로 정렬
frames = sorted(glob.glob(os.path.join(out_dir, "samples_epoch_*.png")))
if len(frames) == 0:
    raise RuntimeError("outputs 폴더에 samples_epoch_*.png가 없어요. 먼저 학습 후 샘플 이미지를 확인하세요!")

# 첫 이미지 사이즈 가져오기
sample = cv2.imread(frames[0])
h, w, _ = sample.shape

# 비디오 라이터 정의 (MP4, fps=2 → 초당 2장)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = 2  # 초당 프레임 수. 더 빠르게 보려면 4~8 정도로 조정 가능
video = cv2.VideoWriter(video_path, fourcc, fps, (w, h))

# 모든 이미지 프레임 추가
for f in frames:
    img = cv2.imread(f)
    video.write(img)

video.release()
print(f"MP4 저장 완료: {video_path}")

# Colab에서 바로 재생
display(Video(video_path, embed=True))

# 다운로드 링크
files.download(video_path)

### “실제 vs 생성” 별도 분류기

### 생성 이미지 뽑아두기

In [ ]:
# 생성 이미지 대량 저장
gen_dir = "/content/gen_images"
os.makedirs(gen_dir, exist_ok=True)

netG.eval() # Generator를 평가 모드로 (BN/Dropout 고정 → 안정적인 출력)
N = 2000  # 필요 수량 (예: 2000장) — Colab 시간에 맞춰 조절
bs = 64
saved = 0 # 저장된 이미지 수 카운트

with torch.no_grad(): # 학습 아님 → gradient 계산 비활성화 (메모리/속도 절약)
    pbar = tqdm(total=N, desc="Generating fake images") # 진행 상황 바 표시
    while saved < N:
        cur = min(bs, N - saved) # 남은 이미지 수가 bs보다 적으면 그만큼만 생성
        z = torch.randn(cur, nz, 1, 1, device=DEVICE) # 랜덤 노이즈 벡터 생성
        fake = netG(z).cpu() # Generator로 가짜 이미지 생성 (CPU로 이동)

        # [-1,1] 범위를 [0,1] 범위로 변환 (이미지 저장용)
        fake = (fake*0.5 + 0.5).clamp(0,1)
        # 한 장씩 PNG로 저장
        for i in range(cur):
            save_image(fake[i], os.path.join(gen_dir, f"fake_{saved+i:05d}.png"))
        saved += cur # 저장된 개수 카운트 업데이트
        pbar.update(cur) # 진행바 업데이트

print("생성 이미지 저장 완료:", gen_dir)

Generating fake images: 100%|██████████| 2000/2000 [00:03<00:00, 697.04it/s]

생성 이미지 저장 완료: /content/gen_images


### 분류 데이터셋/로더
- Images → label 1 (real)
- gen_images → label 0 (fake)

In [ ]:
# Real/Fake 이진 분류용 Dataset
class RealFakeDataset(Dataset):
    def __init__(self, real_root, fake_root, transform):
        # real_root 아래 모든 하위폴더까지 이미지 경로 수집
        self.real = sorted(glob.glob(os.path.join(real_root, "*"))) + \
                    sorted(glob.glob(os.path.join(real_root, "**", "*"), recursive=True))
        # 이미지 확장자만 필터링
        self.real = [p for p in self.real if p.lower().endswith((".jpg",".jpeg",".png",".webp",".bmp"))]
        # fake_root(생성 이미지 폴더)에서 PNG만 수집 (위에서 PNG로 저장했기 때문)
        self.fake = sorted(glob.glob(os.path.join(fake_root, "*.png")))
        self.transform = transform
        # (경로, 라벨) 튜플 리스트 생성: real=1, fake=0  ← 이진 분류 라벨
        self.items = [(p,1) for p in self.real] + [(p,0) for p in self.fake]

    def __len__(self): return len(self.items)
    def __getitem__(self, idx):
        # idx번째 샘플 로드
        p, y = self.items[idx]
        img = Image.open(p).convert("RGB") # RGB 통일(회색/알파 채널 제거)
        img = self.transform(img) # 분류기 입력 전처리 적용
        return img, y # 이미지 텐서, 라벨(int: 0 or 1) 반환

# 분류기용 전처리(224 크기, ImageNet 표준화)
cls_tfm = transforms.Compose([
    transforms.Resize(256), # 짧은 변 256으로 리사이즈
    transforms.CenterCrop(224), # 중앙 224x224로 크롭 (ResNet 등 호환)
    transforms.ToTensor(), # [0,1] 텐서로 변환
    transforms.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225)), # ImageNet 평균/표준편차로 정규화
])

# 전체 Dataset 생성 및 Train/Val 분할
full_ds = RealFakeDataset("/content/Images", "/content/gen_images", cls_tfm)
# 인덱스 셔플 후 8:2로 간단 분할 (층화X, 클래스 불균형 주의)
indices = list(range(len(full_ds)))
random.shuffle(indices)
split = int(0.8*len(indices))
tr_idx, va_idx = indices[:split], indices[split:]
# SubsetRandomSampler로 지정 인덱스만 뽑아 배치 구성
sampler_tr = torch.utils.data.SubsetRandomSampler(tr_idx)
sampler_va = torch.utils.data.SubsetRandomSampler(va_idx)
# DataLoader: 배치/워커/고정메모리 설정
dl_tr = DataLoader(full_ds, batch_size=64, sampler=sampler_tr, num_workers=2, pin_memory=True)
dl_va = DataLoader(full_ds, batch_size=64, sampler=sampler_va, num_workers=2, pin_memory=True)

### 분류기(ResNet18) 학습

In [ ]:
import torchvision.models as models
import torch.nn.functional as F

# 1. 모델 정의 (ResNet18 전이학습)
cls_model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
# 기존 ResNet18은 ImageNet 1000클래스 → 출력층(fc)을 2클래스(Real/Fake)로 교체
cls_model.fc = nn.Linear(cls_model.fc.in_features, 2)  # 2클래스(실제/가짜)
cls_model = cls_model.to(DEVICE)

# 2. 옵티마이저 & 손실 함수
optimizer = optim.Adam(cls_model.parameters(), lr=1e-4) # Adam 최적화기
criterion_ce = nn.CrossEntropyLoss() # 다중 클래스 분류용 손실 (여기선 2클래스)

# 3. 학습 루프 정의
def train_one_epoch():
    cls_model.train()          # 학습 모드 (Dropout/BatchNorm 활성화)
    tot, corr, n = 0.0, 0, 0   # 누적 loss, 정답 개수, 샘플 수
    for x, y in dl_tr:         # 학습 데이터 배치 단위 반복
        x, y = x.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad(set_to_none=True)     # gradient 초기화
        logits = cls_model(x)                     # 모델 forward → 예측 로짓 (B,2)
        loss = criterion_ce(logits, y)            # CrossEntropyLoss 계산
        loss.backward()                           # 역전파 (gradient 계산)
        optimizer.step()                          # weight 업데이트
        tot += loss.item()*y.size(0)              # 배치 손실 합산
        pred = logits.argmax(1)                   # 가장 큰 값이 예측 클래스
        corr += (pred==y).sum().item()            # 정답 개수 누적
        n += y.size(0)                            # 샘플 수 누적
    return tot/n, corr/n                          # 평균 손실, 정확도 반환

@torch.no_grad()
def valid_one_epoch():
    cls_model.eval()         # 평가 모드 (Dropout/BatchNorm 고정)
    tot, corr, n = 0.0, 0, 0
    for x, y in dl_va:       # 검증 데이터 배치 단위 반복
        x, y = x.to(DEVICE), y.to(DEVICE)
        logits = cls_model(x)                     # forward
        loss = criterion_ce(logits, y)            # loss 계산
        tot += loss.item()*y.size(0)              # 손실 합산
        pred = logits.argmax(1)                   # 예측
        corr += (pred==y).sum().item()            # 정답 개수 합산
        n += y.size(0)
    return tot/n, corr/n                          # 평균 손실, 정확도 반환

# 4. 학습 실행
EPOCHS = 5
for ep in range(1, EPOCHS+1):
    tl, ta = train_one_epoch() # train loss / accuracy
    vl, va = valid_one_epoch()  # valid loss / accuracy
    print(f"[CLS {ep:02d}] train {tl:.3f}/{ta:.3f} | valid {vl:.3f}/{va:.3f}")

# 5. 학습된 모델 저장
torch.save(cls_model.state_dict(), os.path.join(OUT_DIR, "real_vs_fake_resnet18.pt"))

[CLS 01] train 0.010/0.999 | valid 0.000/1.000
[CLS 02] train 0.000/1.000 | valid 0.000/1.000
[CLS 03] train 0.000/1.000 | valid 0.000/1.000
[CLS 04] train 0.000/1.000 | valid 0.000/1.000
[CLS 05] train 0.000/1.000 | valid 0.000/1.000


In [ ]:
from PIL import Image
import torch
import torch.nn.functional as F

# 테스트할 이미지 경로 지정 (실제 or 생성)
# test_path = "/content/Images/abraham-manievich_0.jpg"  # 실제
test_path = "/content/gen_images/fake_00000.png"  # 생성

# 1. 이미지 로드 & 전처리
img = Image.open(test_path).convert("RGB") # 이미지 열기 + RGB 통일
img = cls_tfm(img).unsqueeze(0).to(DEVICE)  # 분류기용 전처리(224, 정규화) + 배치 차원 추가 + GPU로 이동
#   - unsqueeze(0): (C,H,W) → (1,C,H,W)로 바꿔서 '배치=1' 형태로 맞춤

# 2. 분류기 추론 (inference)
cls_model.eval()            # 평가 모드 (Dropout/BatchNorm 고정)
with torch.no_grad():       # 추론이므로 gradient 계산 비활성화 (메모리/속도 절약)
    logits = cls_model(img) # 모델 forward → 출력 로짓 (1,2) → [가짜 점수, 실제 점수] # 1 → 배치 크기(batch size) = 1 (이미지 한 장만 넣었으니까), 2 → 클래스 개수(num classes) = 2 (Real vs Fake 분류니까)
    probs = F.softmax(logits, dim=1).cpu().numpy()[0]
    # Softmax로 확률 변환 → numpy 배열로 변환 → [가짜확률, 실제확률]

# 3. 결과 출력
print("실제일 확률:", probs[1])  # 클래스 인덱스 1 = Real
print("가짜일 확률:", probs[0])  # 클래스 인덱스 0 = Fake

실제일 확률: 9.180316e-12
가짜일 확률: 1.0
